In [ ]:
%pip -q install opencv-python pyyaml

### **Azure AI Services**

Login to Azure using the Azure portal and retrieve your service endpoint and key.

- Open Microsoft Edge using the desktop shortcut and open Azure portal using the bookmark.

- Login using the Azure credentials in the Resources tab of the lab guide.

- Inside Azure portal, search for Azure AI Services and select it.

    ![A screenshot showing Azure AI services being searched for in the dropdown](../labguide/img/azureportal_ai_search.png)

- Note the name of the Azure AI service that was precreated for you and then click on the resource.

    ![A screenshot showing an Azure AI service inside a resource group in Azure portal](../labguide/img/azure_ai_service.png)

- You will need the AI service name, language endpoint and one of the key values for the Azure AI Service. You will need these values in the next cell. Review the screenshot and locate these values in your Azure environment, then replace the placeholders in the next cell with your values.

    1. Service name
    2. Key (pick either key 1 or key 2)
    3. Language endpoint

    ![A screenshot showing an Azure AI service overview page in Azure portal](../labguide/img/ai_endpoint_key.png)



### Send a prompt to initialize the model

Replace the placeholders in the code block below endpoint and key you retrieved.

In [ ]:
# Replace these values with the correct values from your Azure AI service
azureAiServiceName = "REPLACE ME WITH YOUR AZURE AI SERVICE NAME" # 1 - Service name
endpoint = ""                                                     # 2 - endpoint
key = "REPLACE ME WITH YOUR AZURE AI SERVICE KEY"                 # 3 - API Key

import os
import requests
import base64
from pprint import pprint

IMAGE_PATH="../media/image/people_on_street.jpg"
ENDPOINT = f"https://{endpoint}openai/deployments/{azureAiServiceName}-gpt-4o-mini-deployment/chat/completions?api-version=2024-08-01-preview"
API_KEY = key

encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are an AI assistant that helps people find information."
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}


# Send request
try:
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
response_json = response.json()
pprint(response_json)

### Describe an image with a prompt

To send an image with our prompt we will convert it to base64 and include it in the payload of our API call.

In [ ]:
import json
from pprint import pprint

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


IMAGE_PATH = "../media/image/columns.png"
image_base64 = image_to_base64(IMAGE_PATH)
payload = {
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Describe this picture:" 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": "data:image/png;base64," + image_base64
                }
            }
        ] } 
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

response = requests.post(ENDPOINT, headers=headers, json=payload)
response_json = response.json()

# Pretty print the JSON response
pprint(response_json)